In [2]:
import cv2
import os
import mediapipe as mp
import numpy as np
from matplotlib import pyplot as plt
import time
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands # Holistic model

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    results=model.process(image)

    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [12]:
def draw_landmarks(image, results):
    if results.multi_hand_landmarks:
        for num, hand in enumerate(results.multi_hand_landmarks):
                    mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                            mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                             )


In [86]:
def extract_keypoints(results):
    #pose = np.array([[res.x, res.y, res.z] for res in results.multi_hand_landmarks]).flatten() if results.multi_hand_landmarks else np.zeros(42*3)
    pose = []
    if results.multi_hand_landmarks:
        for res in results.multi_hand_landmarks :
            print("RES:", res)
            for landmark in res.landmark:
                pose.append(landmark.x)
                pose.append(landmark.y)
                pose.append(landmark.z)
        
        if len(pose)!=126:
            x= len(pose)
            for i in range(0,126-x):
                pose.append(0)
        print("SHAPE:", len(pose))
    else:
        return np.zeros(42*3)
    return pose


In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [80]:
import pandas as pd
def save_as_csv(data_frames, label):
    columns = ['Label', 'Coordinates']
    
    final_df = pd.DataFrame(data_frames, columns=columns)

    directory_path = "MP_Data_csv"
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    csv_file_path = f'{directory_path}.csv'
    final_df.to_csv(csv_file_path, index=False, mode='w')

In [82]:
def save_as_csv2(data_frames, label):
    columns = ['Coordinates', 'Label']
    combined = list(zip(data_frames, label))
    final_df = pd.DataFrame(combined, columns=columns)
    print(final_df)
    directory_path = "MP_Data_csv"
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    csv_file_path = f'{directory_path}.csv'
    final_df.to_csv(csv_file_path, index=False, mode='w')

In [8]:
for action in actions: 
    #dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [87]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    
    # NEW LOOP
    # Loop through actionsqqqqq
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(1, start_folder+1):
            # Loop through video length aka sequence length
        
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, hands)
                
                # Draw landmarks
                draw_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screenqqq
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                if len(keypoints) > 0:
                    np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

RES: landmark {
  x: 0.08860285580158234
  y: 0.637888491153717
  z: 3.0623209568148013e-07
}
landmark {
  x: 0.12766434252262115
  y: 0.6327015161514282
  z: -0.016767973080277443
}
landmark {
  x: 0.16606329381465912
  y: 0.5854154229164124
  z: -0.018772918730974197
}
landmark {
  x: 0.19140499830245972
  y: 0.5512973070144653
  z: -0.01950184255838394
}
landmark {
  x: 0.20672562718391418
  y: 0.5271749496459961
  z: -0.019526563584804535
}
landmark {
  x: 0.1445068120956421
  y: 0.4912319481372833
  z: -0.0020665668416768312
}
landmark {
  x: 0.16113901138305664
  y: 0.4379551410675049
  z: -0.0108438516035676
}
landmark {
  x: 0.17230099439620972
  y: 0.403548926115036
  z: -0.02097901701927185
}
landmark {
  x: 0.18139077723026276
  y: 0.37387505173683167
  z: -0.028253771364688873
}
landmark {
  x: 0.1234060525894165
  y: 0.4778969883918762
  z: -0.0010082839289680123
}
landmark {
  x: 0.14159905910491943
  y: 0.41813188791275024
  z: -0.00855263601988554
}
landmark {
  x: 0.15

In [45]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [46]:
label_map = {label:num for num, label in enumerate(actions)}

In [59]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            if len(res)!=126:
                print("WHAT")
                print(res)
            window.append(res)
        
        sequences.append(window)
        labels.append(label_map[action])

In [83]:
print(len(sequences))
save_as_csv2(sequences,labels)

90
                                          Coordinates  Label
0   [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...      0
1   [[0.19059662520885468, 0.7705538272857666, 1.9...      0
2   [[0.1727619618177414, 0.7667193412780762, 2.15...      0
3   [[0.1813211441040039, 0.7717771530151367, 2.16...      0
4   [[0.1526864916086197, 0.7969017624855042, 2.36...      0
..                                                ...    ...
85  [[0.24393528699874878, 0.8236128091812134, -1....      2
86  [[0.24326014518737793, 0.8286256194114685, -1....      2
87  [[0.24389222264289856, 0.8315956592559814, -1....      2
88  [[0.24230539798736572, 0.8356907963752747, -1....      2
89  [[0.2423851490020752, 0.8319256901741028, -1.6...      2

[90 rows x 2 columns]


In [60]:
X = np.array(sequences)
print(X.shape)

(90, 30, 126)


In [61]:
y = to_categorical(labels).astype(int)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
from sklearn.model_selection import cross_val_score
k_folds = 5
k_choices = [1, 3, 5, 8, 10, 12, 15, 20, 50, 100]
scores = np.zeros((len(k_choices), k_folds))

In [62]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [63]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [67]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,99)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [68]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [66]:
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold, StratifiedKFold
kf = KFold(n_splits = 5, shuffle=True)
acc_per_fold = []
loss_per_fold = []
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')
fold_no = 1
for train, test in kf.split(X, y):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
    model.add(LSTM(128, return_sequences=True, activation='relu'))
    model.add(LSTM(64, return_sequences=False, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(actions.shape[0], activation='softmax'))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    history = model.fit(X[train], y[train], epochs=2000, callbacks=[tb_callback, checkpoint], validation_data=(X[test], y[test]))
    scores = model.evaluate(X[test], y[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    multilabel_confusion_matrix(ytrue, yhat)
    # Increase fold number
    fold_no = fold_no + 1
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')
    

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/2000
2/3 [===================>..........] - ETA: 0s - loss: 1.1019 - categorical_accuracy: 0.2500
Epoch 1: val_categorical_accuracy improved from -inf to 0.27778, saving model to best_model.h5
3/3 [==============================] - 5s 431ms/step - loss: 1.1009 - categorical_accuracy: 0.2639 - val_loss: 1.0871 - val_categorical_accuracy: 0.2778
Epoch 2/2000
2/3 [===================>..........] - ETA: 0s - loss: 1.0854 - categorical_accuracy: 0.4375
Epoch 2: val_categorical_accuracy improved from 0.27778 to 0.77778, saving model to best_model.h5
3/3 [==============================] - 0s 104ms/step - loss: 1.0833 - categorical_accuracy: 0.4861 - val_loss: 1.0458 - val_categorical_accuracy: 0.7778
Epoch 3/2000
2/3 [===================>..........] - ETA: 0s - loss: 1.0527 - categorical_accuracy: 0.7812
Epoch 3: val_categorical_accuracy improved from 0.77778 to 0.94444, saving model to be

In [94]:
print(history)
from sklearn.metrics import confusion_matrix
yhat = model.predict(X[test])
ytrue = np.argmax(y[test], axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
print(confusion_matrix(ytrue, yhat))
print(accuracy_score(ytrue, yhat))

1/1 [==============================] - 0s 66ms/step
[[7 0 0]
 [0 3 1]
 [0 0 7]]
0.9444444444444444


In [69]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 5s 67ms/step - loss: 1.0914 - categorical_accuracy: 0.5059
Epoch 2/2000
3/3 [==============================] - 0s 45ms/step - loss: 1.0909 - categorical_accuracy: 0.5294
Epoch 3/2000
3/3 [==============================] - 0s 52ms/step - loss: 1.0272 - categorical_accuracy: 0.3765
Epoch 4/2000
3/3 [==============================] - 0s 48ms/step - loss: 0.9633 - categorical_accuracy: 0.5412
Epoch 5/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.8463 - categorical_accuracy: 0.6706
Epoch 6/2000
3/3 [==============================] - 0s 50ms/step - loss: 0.7040 - categorical_accuracy: 0.6353
Epoch 7/2000
3/3 [==============================] - 0s 45ms/step - loss: 0.8341 - categorical_accuracy: 0.6588
Epoch 8/2000
3/3 [==============================] - 0s 54ms/step - loss: 0.6896 - categorical_accuracy: 0.7176
Epoch 9/2000
3/3 [==============================] - 0s 55ms/step - loss: 0.6167 - categorical_accuracy: 0.6471
E

In [67]:
res = model.predict(X_test)
actions[np.argmax(res[4])]

NameError: name 'X_test' is not defined

In [44]:
model.save('action.h5')

In [ ]:
del model

In [93]:
model.load_weights('best_model.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('pose.tflite', 'wb') as f:
    f.write(tflite_model)

In [68]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [94]:
yhat = model.predict(X_test)
print(X_test.shape)
print(yhat)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

1/1 [==============================] - 2s 2s/step
(5, 30, 99)
[[0.04229525 0.4575307  0.500174  ]
 [0.01228876 0.54454416 0.44316712]
 [0.00194702 0.62197816 0.37607485]
 [0.06053653 0.45702606 0.4824374 ]
 [0.63916063 0.11915584 0.24168356]]


array([[[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [95]:
accuracy_score(ytrue, yhat)

1.0

In [48]:
from scipy import stats

In [70]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [88]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            
            if res[np.argmax(res)] > threshold: 
                
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('action.h5')

# Convert to TensorFlow Lite without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Apply post-conversion quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()